In [1]:
# Load dependencies
from bs4 import BeautifulSoup
import requests
import json
import time
import random
import webbrowser
import sys

sys.setrecursionlimit(9999)

In [2]:
# Get the function parameters from terminal
# url = sys.argv[1]
# category = int(sys.argv[2])

In [3]:
# Load url
url = "https://www.ticketswap.com/event/queen-adam-lambert-the-rhapsody-tour/23a41750-3429-4ae9-9727-2ec36f43bf4f"

In [4]:
def check_ticketswap_offers(url, category, counter = 0):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    ldjson_soup = soup.findAll("script", attrs = {'type':"application/json"})
    ldjson_str = ldjson_soup[0].text
    ldjson = json.loads(ldjson_str)
    # Check it there are any available tickets:
    if ldjson['props']['pageProps']['event']['availableTicketsCount'] < 1:
        seconds = random.randrange (75, 125) / 100
        time.sleep(seconds)
        counter += 1
        print("Try %s" % counter)
        # If so, return the avaiable ticket (listing) url:
        return check_ticketswap_offers(url, category, counter)
    # Open browser with url (+ category index)
    url_open = ldjson['props']['pageProps']['data']['node']['event']['types']['edges'][category-1]['node']['availableListings']['edges'][0]['node']['uri']['url']
    webbrowser.open_new(url_open)
    
#Tip: use ldjson.keys() to see set of keys on the highest level

In [12]:
#TODO: come up with a way to scan ticket types and select via prompt to create dynamic code

In [22]:
# Load & test url:
check_ticketswap_offers(url, 1)

Try 1
Try 2
Try 3
Try 4
Try 5
Try 6
Try 7
Try 8
Try 9
Try 10
Try 11
Try 12
Try 13
Try 14
Try 15
Try 16
Try 17
Try 18
Try 19
Try 20
Try 21
Try 22
Try 23
Try 24
Try 25
Try 26
Try 27


Top-page research:

In [33]:
# url = 'https://www.ticketswap.com/event/the-weeknd-the-after-hours-tour/ea3cfc09-a4c7-44d1-afb3-07aac5e14551'
url = 'https://www.ticketswap.com/event/pina-bausch-die-sieben-todsunden/regular-tickets/b186c0be-2138-4ea1-b66a-82c9848678b4/1623976'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
ldjson_soup = soup.findAll("script", attrs = {'type':"application/json"})
ldjson_str = ldjson_soup[0].text
ldjson = json.loads(ldjson_str)

In [34]:

urls = [ldjson['props']['apolloState'][x]['url'] for x in ldjson['props']['apolloState'].keys() if x.endswith('.uri')]

In [35]:
ticketUrls = [url for url in urls if 'utm_source=ticketswap' in url]
print(ticketUrls)

[]


In [36]:
for i in ldjson['props']['apolloState']:
    print(i)

RXZlbnRUeXBlOjE2MjM5NzY=
VXNlcjo1MTAxNDU4
$RXZlbnRUeXBlOjE2MjM5NzY=.listings({"filter":{"listingStatus":"AVAILABLE"},"first":2}).edges.0.node
$RXZlbnRUeXBlOjE2MjM5NzY=.listings({"filter":{"listingStatus":"AVAILABLE"},"first":2}).edges.0
$RXZlbnRUeXBlOjE2MjM5NzY=.listings({"filter":{"listingStatus":"AVAILABLE"},"first":2})
$RXZlbnRUeXBlOjE2MjM5NzY=.listings({"filter":{"listingStatus":"SOLD"},"first":2})
$RXZlbnRUeXBlOjE2MjM5NzY=.ticketAlerts({"first":8})
RXZlbnQ6YjE4NmMwYmUtMjEzOC00ZWExLWI2NmEtODJjOTg0ODY3OGI0
$RXZlbnQ6YjE4NmMwYmUtMjEzOC00ZWExLWI2NmEtODJjOTg0ODY3OGI0.seoMetadata
TG9jYXRpb246MzI3ODQ=
Q2l0eTo0MDM=
$TG9jYXRpb246MzI3ODQ=.country
$RXZlbnQ6YjE4NmMwYmUtMjEzOC00ZWExLWI2NmEtODJjOTg0ODY3OGI0.types({"first":99}).edges.0
$RXZlbnQ6YjE4NmMwYmUtMjEzOC00ZWExLWI2NmEtODJjOTg0ODY3OGI0.types({"first":99})
ROOT_QUERY
$RXZlbnRUeXBlOjE2MjM5NzY=.availableListings.edges.0.node.price.originalPrice
$RXZlbnRUeXBlOjE2MjM5NzY=.availableListings.edges.0.node.price
$RXZlbnRUeXBlOjE2MjM5NzY=.availableL

In [42]:
ldjson['props']['apolloState']['$TGlzdGluZzo0NjcyNDIw.price']

{'totalPriceWithTransactionFee': {'type': 'id',
  'generated': True,
  'id': '$TGlzdGluZzo0NjcyNDIw.price.totalPriceWithTransactionFee',
  'typename': 'Money'},
 '__typename': 'ListingPrice'}

Sub-page research:

In [26]:
url = 'https://www.ticketswap.com/event/the-weeknd-the-after-hours-tour/upper-rank/ea3cfc09-a4c7-44d1-afb3-07aac5e14551/1615090'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
ldjson_soup = soup.findAll("script", attrs = {'type':"application/json"})
ldjson_str = ldjson_soup[0].text
ldjson = json.loads(ldjson_str)

In [27]:
ldjson.keys()

dict_keys(['props', 'page', 'query', 'buildId', 'assetPrefix', 'runtimeConfig', 'isFallback'])

In [28]:
ldjson['props']

ilableListings.edges.0.node.price': {'originalPrice': {'type': 'id',
    'generated': True,
    'id': '$RXZlbnRUeXBlOjE2MTUwOTI=.availableListings.edges.0.node.price.originalPrice',
    'typename': 'Money'},
   '__typename': 'ListingPrice'},
  '$RXZlbnRUeXBlOjE2MTUwOTI=.availableListings.edges.0.node': {'price': {'type': 'id',
    'generated': True,
    'id': '$RXZlbnRUeXBlOjE2MTUwOTI=.availableListings.edges.0.node.price',
    'typename': 'ListingPrice'},
   '__typename': 'PublicListing'},
  '$RXZlbnRUeXBlOjE2MTUwOTI=.availableListings.edges.0': {'node': {'type': 'id',
    'generated': True,
    'id': '$RXZlbnRUeXBlOjE2MTUwOTI=.availableListings.edges.0.node',
    'typename': 'PublicListing'},
   '__typename': 'ListingEdge'},
  '$RXZlbnRUeXBlOjE2MTUwOTI=.availableListings': {'edges': [{'type': 'id',
     'generated': True,
     'id': '$RXZlbnRUeXBlOjE2MTUwOTI=.availableListings.edges.0',
     'typename': 'ListingEdge'}],
   '__typename': 'ListingConnection'},
  '$RXZlbnRUeXBlOjE2MTUwO